In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, explained_variance_score, mean_absolute_error, mean_squared_log_error, median_absolute_error, r2_score
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [6]:
def MASE(training_series, testing_series, prediction_series, m=365):
    n = training_series.shape[0]
    d = np.abs(  np.diff( training_series) ).sum()/(n-1)
#     i=0
#     d = 0
#     while (i+m) < n:
#         d += np.abs(training_series[i]-training_series[i+m])
#         i += 1
#     d = d/(n-m)
    d1 = np.abs(training_series).mean() 
    errors = np.abs(testing_series - prediction_series)
    return errors.mean()/d
#     return errors.mean()/d1

def MAPE(y_true, y_pred):
    mape = 100*sum(np.divide(np.abs(y_true-y_pred),y_true))/len(y_true)
    return mape[0]

def next_batch(training_data,batch_size,steps,start_point):
    # Grab a random starting point for each batch
    # rand_start = np.random.randint(0,len(training_data)-steps*2) 
    rand_start = start_point
    # Create Y data for time series in the batches
    y_batch = np.array(training_data[rand_start:rand_start+steps+1]).reshape(1,steps+1)
    X_batch = np.array(training_data[rand_start:rand_start+steps]).reshape(-1, steps, 1)
    y_batch = np.array(training_data[rand_start+steps:rand_start+steps*2]).reshape(-1, steps, 1)
    start_point += steps
    if (start_point + 2*steps) >= len(training_data):
        start_point = 0
    return X_batch, y_batch, start_point

In [ ]:
datasetnames=["coast","east","ercot","far_west","north","north_c","south_c","southern","west"]

#datasetnames=["coast"]

for dtindx in range(len(datasetnames)):

    datasetName = datasetnames[dtindx]
    train = pd.read_csv('../data/Daily_Load_Data/train/'+datasetName+'.csv',index_col='Time')
    test = pd.read_csv('../data/Daily_Load_Data/test/'+datasetName+'.csv',index_col='Time')
    train = train.dropna(axis=0)
    test = test.dropna(axis=0)

    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train)

    # Just one feature, the time series
    num_inputs = 1
    # Num of steps in each batch
    num_time_steps = 365
    # 100 neuron layer, play with this
    num_neurons = 1500
    # number of layer
    num_layers = 3
    # Just one output, predicted time series
    num_outputs = 1
    ## You can also try increasing iterations, but decreasing learning rate
    # learning rate you can play with this
    learning_rate = 0.001
    # how many iterations to go through (training steps), you can play with this
    num_train_iterations = 1000
    # Size of the batch of data
    batch_size = 1

    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, [None, num_time_steps, num_inputs])
    y = tf.placeholder(tf.float32, [None, num_time_steps, num_outputs])

    cell = tf.contrib.rnn.OutputProjectionWrapper(tf.nn.rnn_cell.MultiRNNCell(
        [tf.nn.rnn_cell.GRUCell(num_units=num_neurons, activation=tf.nn.relu) 
                                        for layer in range(num_layers)]),
                                        output_size=num_outputs)
#     cell = tf.contrib.rnn.OutputProjectionWrapper(tf.nn.rnn_cell.MultiRNNCell(
#         [tf.nn.rnn_cell.LSTMCell(num_units=num_neurons, activation=tf.nn.relu) 
#                                         for layer in range(num_layers)]),
#                                         output_size=num_outputs)
    outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    loss = tf.reduce_mean(tf.square(outputs - y)) # MSE
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train = optimizer.minimize(loss)
    init = tf.global_variables_initializer()


    model_file_name = "../../temp/model_"+datasetName+".ckpt"
    saver = tf.train.Saver()
    start_point = 0
    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
        sess.run(init)
#         saver.restore(sess, model_file_name)
        for iteration in range(num_train_iterations+1):
            X_batch, y_batch, start_point = next_batch(train_scaled,batch_size,num_time_steps,start_point)
            sess.run(train, feed_dict={X: X_batch, y: y_batch})
            if iteration % 1000 == 0 and iteration != 0:
                try:
                    #mse = loss.eval(feed_dict={X: X_batch, y: y_batch})
                    #print(iteration, "\tMSE:", mse)
                    
                    # calculate train error
                    s_point = 0
                    mse = 0
                    mae = 0
                    mase = 0
                    mape = 0
                    trun = 10
                    for i in range(trun):
                        X_batch, y_batch, s_point = next_batch(train_scaled,batch_size,num_time_steps,s_point)
                        y_pred = sess.run(outputs, feed_dict={X: X_batch})
                        results = scaler.inverse_transform(y_pred.reshape(num_time_steps,1))
                        true_data = y_batch.reshape(num_time_steps,1)
                        true_data = scaler.inverse_transform(true_data)
                        n = min(results.shape[0],true_data.shape[0])

                        y_pred = results[:n]
                        y_true = true_data[:n]

                        mse += mean_squared_error(y_true=y_true,y_pred=y_pred)
                        mae += mean_absolute_error(y_true=y_true,y_pred=y_pred)

                        train_series = scaler.inverse_transform(train_scaled)
                        mase += MASE(train_series.ravel(),y_true.ravel(),y_pred.ravel())
                        mape += MAPE(y_true, y_pred)
                    mse /= trun
                    mae /= trun 
                    mase /= trun
                    mape /= trun
                    print(iteration," train errors:\t"+datasetName+'\tRMSE:', round(mse**0.5,4), '\tMAE:', 
                          round(mae,4), '\tMASE:', round(mase,4), '\tMAPE:', round(mape,4))
                    
                    #test error
                    train_seed = list(train_scaled[-num_time_steps:])
                    X_batch = np.array(train_seed[-num_time_steps:]).reshape(1, num_time_steps, 1)
                    y_pred = sess.run(outputs, feed_dict={X: X_batch})
                    results = scaler.inverse_transform(y_pred.reshape(num_time_steps,1))
                    true_data = test.values
                    n = min(results.shape[0],true_data.shape[0])

                    y_pred = results[:n]
                    y_true = true_data[:n]

                    mse = mean_squared_error(y_true=y_true,y_pred=y_pred)
                    mae = mean_absolute_error(y_true=y_true,y_pred=y_pred)

                    train_series = scaler.inverse_transform(train_scaled)
                    mase = MASE(train_series.ravel(),y_true.ravel(),y_pred.ravel())
                    mape = MAPE(y_true, y_pred)

                    print(iteration," test  errors:\t"+datasetName+'\tRMSE:', round(mse**0.5,4), '\tMAE:', 
                          round(mae,4), '\tMASE:', round(mase,4), '\tMAPE:', round(mape,4))
                    saver.save(sess, model_file_name)
                except Exception as e:
                    print("error1")
                    break

1000  train errors:	coast	RMSE: 964.56 	MAE: 754.941 	MASE: 1.3737 	MAPE: 6.9901
1000  test  errors:	coast	RMSE: 1272.8966 	MAE: 995.0476 	MASE: 1.8106 	MAPE: 8.1972
1000  train errors:	east	RMSE: 148.9322 	MAE: 115.5939 	MASE: 1.3297 	MAPE: 8.2951
1000  test  errors:	east	RMSE: 193.1373 	MAE: 144.7921 	MASE: 1.6655 	MAPE: 9.0942
1000  train errors:	ercot	RMSE: 3637.0805 	MAE: 2812.4913 	MASE: 1.5466 	MAPE: 7.2611
1000  test  errors:	ercot	RMSE: 5050.4834 	MAE: 3772.263 	MASE: 2.0744 	MAPE: 8.2104


In [ ]:
# sum(abs(y_pred-y_true))/len(y_true)
# plt.plot(y_true.ravel())
# plt.plot(y_pred.ravel())

In [ ]:
# # d = np.abs(np.diff(y_true)).sum()
# # print(d)
# d =(np.diff(y_true.ravel()))
# d= sum(abs(d))/len(d)
# print(d)

In [ ]:
# try:
#             # calculate train error
#             start_point = 0
#             mse = 0
#             mae = 0
#             mase = 0
#             mape = 0
#             trun = 10
#             for i in range(trun):
#                 X_batch, y_batch, start_point = next_batch(train_scaled,batch_size,num_time_steps,start_point)
#                 y_pred = sess.run(outputs, feed_dict={X: X_batch})
#                 results = scaler.inverse_transform(y_pred.reshape(num_time_steps,1))
#                 true_data = y_batch.reshape(num_time_steps,1)
#                 true_data = scaler.inverse_transform(true_data)
#                 n = min(results.shape[0],true_data.shape[0])

#                 y_pred = results[:n]
#                 y_true = true_data[:n]

#                 mse += mean_squared_error(y_true=y_true,y_pred=y_pred)
#                 mae += mean_absolute_error(y_true=y_true,y_pred=y_pred)

#                 train_series = scaler.inverse_transform(train_scaled)
#                 mase += MASE(train_series.ravel(),y_true.ravel(),y_pred.ravel())
#                 mape += MAPE(y_true, y_pred)
#             mse /= trun
#             mae /= trun 
#             mase /= trun
#             mape /= trun
#             print("train errors:\t"+datasetName+'\tRMSE:', round(mse**0.5,4), '\tMAE:', 
#                   round(mae,4), '\tMASE:', round(mase,4), '\tMAPE:', round(mape,4))

#             # calculate test error
#             train_seed = list(train_scaled[-num_time_steps:])
#             X_batch = np.array(train_seed[-num_time_steps:]).reshape(1, num_time_steps, 1)
#             y_pred = sess.run(outputs, feed_dict={X: X_batch})
#             results = scaler.inverse_transform(y_pred.reshape(num_time_steps,1))
#             true_data = test.values
#             n = min(results.shape[0],true_data.shape[0])

#             y_pred = results[:n]
#             y_true = true_data[:n]

#             mse = mean_squared_error(y_true=y_true,y_pred=y_pred)
#             mae = mean_absolute_error(y_true=y_true,y_pred=y_pred)

#             train_series = scaler.inverse_transform(train_scaled)
#             mase = MASE(train_series.ravel(),y_true.ravel(),y_pred.ravel())
#             mape = MAPE(y_true, y_pred)

#             print('test errors:\t'+datasetName+'\tRMSE:', round(mse**0.5,4), '\tMAE:', 
#                   round(mae,4), '\tMASE:', round(mase,4), '\tMAPE:', round(mape,4))

#         except Exception as e:
#             print("error2")
 
    
    
    
# prediction
    #train_seed = list(train_scaled[-num_time_steps:])
    
    ## Now create a for loop that 
#     for iteration in range(num_time_steps):
#         X_batch = np.array(train_seed[-num_time_steps:]).reshape(1, num_time_steps, 1)
#         y_pred = sess.run(outputs, feed_dict={X: X_batch})
#         train_seed.append(y_pred[0, -1, 0])
#     results = scaler.inverse_transform(np.array(train_seed[num_time_steps:]).reshape(num_time_steps,1))
    
    #X_batch = np.array(train_seed[-num_time_steps:]).reshape(1, num_time_steps, 1)
    #y_pred = sess.run(outputs, feed_dict={X: X_batch})
    #results = scaler.inverse_transform(y_pred.reshape(num_time_steps,1))
    #true_data = test.values
    #n = min(results.shape[0],true_data.shape[0])
    #y_pred = results[:n]
    #y_true = true_data[:n]
    
    #mse = mean_squared_error(y_true=y_true,y_pred=y_pred)
    #print('RMSE:', mse**0.5)
#     evs = explained_variance_score(y_true=y_true,y_pred=y_pred)
#     print('EVS:', evs)
    #mae = mean_absolute_error(y_true=y_true,y_pred=y_pred)
    #print('MAE:', mae)
#     msle = mean_squared_log_error(y_true=y_true,y_pred=y_pred)
#     print('MSLE:', msle)
#     meae = median_absolute_error(y_true=y_true,y_pred=y_pred)
#     print('MEAE:', meae)
#     r2 = r2_score(y_true=y_true,y_pred=y_pred)
#     print('R2-Score:', r2)

In [ ]:
y_truedf = pd.DataFrame(data=y_true)
y_preddf = pd.DataFrame(data=y_pred)

In [ ]:
y_truedf.to_csv("y_true.csv")
y_preddf.to_csv("y_pred.csv")

In [ ]:
MASE(train_series.ravel(),y_true.ravel(),y_pred.ravel(),m=1)